# Get Embeddings

In [32]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
!pip install openai
!pip install load_dotenv
!pip install azure-identity
!pip install azure-mgmt-cognitiveservices


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


## Set up Azure OpenAI

In [69]:
import os
import openai
from dotenv import load_dotenv
from openai import AzureOpenAI

# Set up Azure OpenAI
#load_dotenv()
# api_type = "OpenAI"
# api_base = "https://tsisodia-openai.openai.azure.com/" # Api base is the 'Endpoint' which can be found in Azure Portal where Azure OpenAI is created. It looks like https://xxxxxx.openai.azure.com/
# api_version = "2022-12-01"
# api_key = "cb874776dc4046098e28775b0d6494d0"


# Set up Azure OpenAI
# load_dotenv()
# openai.api_type = "azure"
# openai.api_base = "https://tsisodia-openai.openai.azure.com/" # Api base is the 'Endpoint' which can be found in Azure Portal where Azure OpenAI is created. It looks like https://xxxxxx.openai.azure.com/
# openai.api_version = "2022-12-01"
# openai.api_key = os.getenv("OPENAI_API_KEY")

## Load Data

In [81]:
import pandas as pd

df_orig = pd.read_csv("bbc-news-data.csv", delimiter='\t')
df = df_orig.copy()
df

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...
...,...,...,...,...
2220,tech,397.txt,BT program to beat dialler scams,BT is introducing two initiatives to help bea...
2221,tech,398.txt,Spam e-mails tempt net shoppers,Computer users across the world continue to i...
2222,tech,399.txt,Be careful how you code,A new European directive could put software w...
2223,tech,400.txt,US cyber security chief resigns,The man making sure US computer networks are ...


## Deploy a model

In [58]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient

os.environ['AZURE_CLIENT_ID']="abc"
os.environ['AZURE_TENANT_ID']="abc"
os.environ['AZURE_CLIENT_SECRET']="abc"

# list models deployed with embeddings capability
deployment_id = None
client = CognitiveServicesManagementClient(
        credential=DefaultAzureCredential(),
        subscription_id="abc",
    )

response = client.deployments.begin_create_or_update(
        resource_group_name="cloud-shell-storage-eastus",
        account_name="tsisodia-openai",
        deployment_name="tsisodia-gpt",
        deployment={
            "properties": {"model": {"format": "OpenAI", "name": "text-embedding-ada-002", "version": "2"}},
            "sku": {"capacity": 2, "name": "Standard"},
        },
    )

## Get Embeddings
ref: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/tutorials/embeddings?tabs=bash

In [84]:
# Use the latest text embedding model
import os
import openai
from dotenv import load_dotenv
from openai import AzureOpenAI

client = AzureOpenAI(
  api_key = "xyz",
  api_version = "2023-05-15",
  azure_endpoint = "https://tsisodia-xyz.openai.azure.com/"  
)
embedding_response = client.embeddings.create(
    input="Hello",
    model='tsisodia-gpt'
    )
embedding = embedding_response.data[0].embedding

print("Embedding the text:", embedding)

# Access embeddings
# len(embedding["data"][0]["embedding"])

Embedding the text: [-0.021819502115249634, -0.007147768046706915, -0.028617016971111298, -0.02436208352446556, -0.023570770397782326, 0.029084021225571632, -0.01248589251190424, -0.00280202878639102, -0.00822447333484888, -0.0051078652031719685, 0.029317524284124374, -0.0032560613472014666, -0.01567060686647892, -0.0025928495451807976, 0.012174556031823158, -0.0009461712324991822, 0.0385797880589962, 0.005762969143688679, 0.019004501402378082, -0.013672863133251667, -0.01980878785252571, 0.010098978877067566, 0.0050689480267465115, 0.008808230049908161, -0.008127180859446526, -0.005315423011779785, 0.002570148091763258, -0.012122666463255882, 0.0033857848029583693, -0.015722494572401047, 0.0038106294814497232, -0.016176527366042137, -0.017849961295723915, -0.012926952913403511, 0.004057104233652353, -0.016306251287460327, -0.0009186049574054778, -0.009768184274435043, 0.021508166566491127, -0.008406086824834347, 0.012965870089828968, -0.006460233125835657, 0.003580370219424367, -0.006

In [85]:
df['embedding'] = ''

for i in range(len(df)):    
#for i in range(760,765):
    try:
        embedding = openai.embeddings.create(input=df['content'][i], deployment_id=deployment_id)
        df['embedding'][i] = embedding['data'][0]['embedding']
    except Exception as err:
        i
        print(f"Unexpected {err=}, {type(err)=}")

0

Unexpected err=APIRemovedInV1('\n\nYou tried to access openai.Embedding, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.\n\nYou can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. \n\nAlternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`\n\nA detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742\n'), type(err)=<class 'openai.lib._old_api.APIRemovedInV1'>


1

Unexpected err=APIRemovedInV1('\n\nYou tried to access openai.Embedding, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.\n\nYou can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. \n\nAlternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`\n\nA detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742\n'), type(err)=<class 'openai.lib._old_api.APIRemovedInV1'>


2

...

Unexpected err=APIRemovedInV1('\n\nYou tried to access openai.Embedding, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.\n\nYou can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. \n\nAlternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`\n\nA detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742\n'), type(err)=<class 'openai.lib._old_api.APIRemovedInV1'>


2223

Unexpected err=APIRemovedInV1('\n\nYou tried to access openai.Embedding, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.\n\nYou can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. \n\nAlternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`\n\nA detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742\n'), type(err)=<class 'openai.lib._old_api.APIRemovedInV1'>


2224

Unexpected err=APIRemovedInV1('\n\nYou tried to access openai.Embedding, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.\n\nYou can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. \n\nAlternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`\n\nA detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742\n'), type(err)=<class 'openai.lib._old_api.APIRemovedInV1'>


## Save embeddings
Following notebooks will require embeddings generated in this notebook. 

In [7]:
# Save embeddings
df.to_csv("../data/bbc-news-data-embedding.csv", sep='\t', index=False)